In [59]:
import logging
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from indexers import msc

logger = logging.getLogger('indexing')
hdlr = logging.FileHandler('/var/tmp/wimcat.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.WARNING)




IndentationError: expected an indented block (msc.py, line 17)

In [60]:
def indexerMSC(containerID, driver, logger):
    transforms = {
        "date" : [],
        "status" : [],
        "location" : [],
        "transport" : [],
        "voyage" : []        
    }
    
    def try_element_by_xpath(webelement, xpath):
        try:
            result = webelement.find_element_by_xpath(xpath)
            return result.text
        except:
            return ""

    xpaths = {
        'status' : './/td[2]/span',
        'date' : './/td[3]/span',
        'location' : './/td[1]/span',
        'transport' : './/td[4]/span',
        'voyage' : './/td[5]/span',
        }
    row_xp = '//*[@id="ctl00_ctl00_plcMain_plcMain_rptBOL_ctl00_rptContainers_ctl01_pnlContainer"]/table[2]/tbody/tr[*]'
    url = 'https://www.msc.com/track-a-shipment?agencyPath=deu'
    input_xpath = '//*[@id="ctl00_ctl00_plcMain_plcMain_TrackSearch_txtBolSearch_TextField"]'

    try:
        driver.get(url)
        elem = driver.find_element_by_xpath(input_xpath)
        elem.clear()
        elem.send_keys(containerID)
        elem.send_keys(Keys.RETURN)
        rows = driver.find_elements_by_xpath(row_xp)
    except Exception as e:
        print "> Error: {}".format(e)
        return [], False
    events = []

    for ro in rows:
        events.append({fi:try_element_by_xpath(ro, xpaths[fi]) for fi in xpaths.keys()})
    return events, True
    
    
    

In [61]:
driver = webdriver.Chrome()
t = indexerMSC("MSCU6713026", driver, logger)

In [62]:
t

([{'date': u'02/11/2017',
   'location': u'COLOMBO, 11, LK',
   'status': u'Estimated Time of Arrival',
   'transport': u'MSC SOLA',
   'voyage': u' '},
  {'date': u'07/10/2017',
   'location': u'HAMBURG, HH, DE',
   'status': u'Loaded',
   'transport': u'MSC SOLA',
   'voyage': u'IS740A'},
  {'date': u'05/10/2017',
   'location': u'HAMBURG, HH, DE',
   'status': u'Unloaded From Truck',
   'transport': u' ',
   'voyage': u' '},
  {'date': u'05/10/2017',
   'location': u'BAD SALZUFLEN, NW, DE',
   'status': u'Empty to Shipper',
   'transport': u' ',
   'voyage': u' '},
  {'date': u'05/10/2017',
   'location': u'BAD SALZUFLEN, NW, DE',
   'status': u'Empty to Shipper',
   'transport': u' ',
   'voyage': u' '}],
 True)

In [53]:
def toASCII(val):
    return "a:{}".format(val)
    
def dates_slash_dot(val):
    return val.replace("/", ".")

def dates_YY_YYYY(val):
    return "haha, {}".format(val)
    

transforms = {
    "date" : [dates_slash_dot, dates_YY_YYYY],
    "status" : [b, c, c]
}


In [48]:
q = transforms["date"][0]("blub")
q

'a:blub'

In [54]:
q = "25/13/1999"
for func in transforms["date"]:
    q = func(q)
print q

haha, 25.13.1999


In [57]:
for k in transforms.iterkeys():
    print k

date
status
